In [1]:
library(Seurat)
library(Signac)
library(magrittr)
library(ggplot2)

Loading required package: SeuratObject

Loading required package: sp

‘SeuratObject’ was built under R 4.3.2 but the current version is
4.3.3; it is recomended that you reinstall ‘SeuratObject’ as the ABI
for R may have changed

‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.5; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect




Import seurat object

In [2]:
obj.seurat = readRDS("/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/2.seurat_pipeline.240506/obj.seurat.rds")

Define merged cell types

In [3]:
obj.seurat$cell_type.merged = NA

In [4]:
obj.seurat@meta.data[obj.seurat$cell_type.rename %in% c("CD4_pos_T_activated",
                                                        "CD4_pos_T_naive",
                                                        "CD8_pos_T",
                                                        "CD8_pos_T_naive"),
                     "cell_type.merged"] = "T"
obj.seurat@meta.data[obj.seurat$cell_type.rename %in% c("Transitional_B",
                                                        "B1_B",
                                                        "Naive_CD20_pos_B"),
                     "cell_type.merged"] = "B"
obj.seurat@meta.data[obj.seurat$cell_type.rename %in% c("CD14_pos_Mono",
                                                        "CD16_pos_Mono",
                                                        "ID2_hi_myeloid_prog",
                                                        "G_M_prog"),
                     "cell_type.merged"] = "Myeloid"
obj.seurat@meta.data[obj.seurat$cell_type.rename %in% c("cDC2",
                                                        "pDC"),
                     "cell_type.merged"] = "Dendritic"
obj.seurat@meta.data[obj.seurat$cell_type.rename %in% c("Erythroblast",
                                                        "Normoblast",
                                                        "Proerythroblast"),
                     "cell_type.merged"] = "Erythroid"
table(obj.seurat$cell_type.merged)


        B Dendritic Erythroid   Myeloid         T 
     9752      2050      8996     14048     22525 

Generate fragment files for each merged cell type

In [7]:
dir_path.frag_split = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/3.merge_cell_type.240522/fragments_split"
dir.create(dir_path.frag_split,recursive = T)

Signac::SplitFragments(obj.seurat,
                      assay = "ATAC_empty",
                      group.by = "cell_type.merged",
                      append = F,
                      outdir = dir_path.frag_split)

Processing file /maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor1/outs/atac_fragments.tsv.gz




Processing file /maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor2/outs/atac_fragments.tsv.gz




Processing file /maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site1_donor3/outs/atac_fragments.tsv.gz




Processing file /maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site2_donor1/outs/atac_fragments.tsv.gz




Processing file /maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240430/neurips2021_BMMC/cellranger_res/site2_donor4/outs/atac_fragments.tsv.gz




Processing file /maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/10x_multiome_240

In [10]:
dir_path.frag = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/3.merge_cell_type.240522/fragments"
dir.create(dir_path.frag,recursive = T)
names.sample = c("s1d1",
                 "s1d2",
                 "s1d3",
                 "s2d1",
                 "s2d4",
                 "s2d5",
                 "s3d10",
                 "s3d3",
                 "s3d6",
                 "s3d7",
                 "s4d1",
                 "s4d8",
                 "s4d9")

for(name.cell_type in unique(obj.seurat$cell_type.merged[!is.na(obj.seurat$cell_type.merged)])){
  df.frag = data.frame(
    Column1 = character(0),
    Column2 = integer(0),
    Column3 = integer(0),
    Column4 = character(0),
    Column5 = integer(0)
  )
  
  for (n.tmp in 1:length(names.sample)) {
    cat(name.cell_type,n.tmp,"\n")
    
    path.frag.tmp = paste(dir_path.frag_split,
                          "/",
                          name.cell_type,
                          ".",
                          n.tmp,
                          ".bed",
                          sep = "")
    if (file.info(path.frag.tmp)$size > 0) {
      df.frag.tmp = 
        read.delim(path.frag.tmp,
                   header = F)
      df.frag.tmp[,4] = paste(names.sample[n.tmp],
                              df.frag.tmp[,4],
                              sep = "_")
      df.frag = rbind(df.frag,
                      df.frag.tmp)
    }
  }
  
  df.frag = df.frag[order(df.frag[,2]),]
  df.frag = df.frag[order(df.frag[,1]),]
  
  write.table(df.frag,
              paste(dir_path.frag,"/",name.cell_type,".","atac_fragments.tsv",sep = ""),
              col.names = F,
              row.names = F,
              quote = F,
              sep = "\t")
  remove(df.frag)
  
  Rsamtools::bgzip(paste(dir_path.frag,"/",name.cell_type,".","atac_fragments.tsv",sep = ""),
                   paste(dir_path.frag,"/",name.cell_type,".","atac_fragments.tsv.gz",sep = ""))
  
  Rsamtools::indexTabix(paste(dir_path.frag,"/",name.cell_type,".","atac_fragments.tsv.gz",sep = ""),
                        format = "bed")
  system(paste("rm -f",paste(dir_path.frag,"/",name.cell_type,".","atac_fragments.tsv",sep = "")))
}

Warning message in dir.create(dir_path.frag, recursive = T):
“'/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/3.merge_cell_type.240522/fragments' already exists”


B 1 
B 2 
B 3 
B 4 
B 5 
B 6 
B 7 
B 8 
B 9 
B 10 
B 11 
B 12 
B 13 
Myeloid 1 
Myeloid 2 
Myeloid 3 
Myeloid 4 
Myeloid 5 
Myeloid 6 
Myeloid 7 
Myeloid 8 
Myeloid 9 
Myeloid 10 
Myeloid 11 
Myeloid 12 
Myeloid 13 
T 1 
T 2 
T 3 
T 4 
T 5 
T 6 
T 7 
T 8 
T 9 
T 10 
T 11 
T 12 
T 13 
Erythroid 1 
Erythroid 2 
Erythroid 3 
Erythroid 4 
Erythroid 5 
Erythroid 6 
Erythroid 7 
Erythroid 8 
Erythroid 9 
Erythroid 10 
Erythroid 11 
Erythroid 12 
Erythroid 13 
Dendritic 1 
Dendritic 2 
Dendritic 3 
Dendritic 4 
Dendritic 5 
Dendritic 6 
Dendritic 7 
Dendritic 8 
Dendritic 9 
Dendritic 10 
Dendritic 11 
Dendritic 12 
Dendritic 13 


Generate RNA count matrix for each cell type

In [13]:
dir_path.rna_matrix = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/3.merge_cell_type.240522/RNA_count_matrix/"
dir.create(dir_path.rna_matrix,recursive = T)

for(name.cell_type in unique(obj.seurat$cell_type.merged[!is.na(obj.seurat$cell_type.merged)])){
  matrix.tmp = 
    obj.seurat@assays$RNA$count[,!is.na(obj.seurat$cell_type.merged) & obj.seurat$cell_type.merged == name.cell_type]
  write.csv(as.data.frame(matrix.tmp),
            gzfile(paste(dir_path.rna_matrix,
                         "/",
                         name.cell_type,
                         ".rna_count_matrix.csv.gz",
                        sep = "")),
            quote = F)
}

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 4.6 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 6.6 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 10.5 GiB”
Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 4.2 GiB”


In [11]:
df.cell_type = data.frame(cell_type = names(table(obj.seurat$cell_type.merged)),
                          cell_number = table(obj.seurat$cell_type.merged),
                          total_fragment_count = NA,
                          total_UMI_count = NA)
rownames(df.cell_type) = df.cell_type$cell_type

for(cell_type.tmp in df.cell_type$cell_type){
    cell_names.tmp = colnames(obj.seurat)[!is.na(obj.seurat@meta.data$cell_type.merged) & obj.seurat@meta.data$cell_type.merged == cell_type.tmp]
    df.cell_type[cell_type.tmp,"total_fragment_count"] = sum(obj.seurat@meta.data[cell_names.tmp,"atac_fragments"])
    df.cell_type[cell_type.tmp,"total_UMI_count"] = sum(obj.seurat@meta.data[cell_names.tmp,"gex_umis_count"])
}

df.cell_type$mean_fragment_count = round(df.cell_type$total_fragment_count / df.cell_type$cell_number.Freq,0)
df.cell_type$mean_UMI_count = round(df.cell_type$total_UMI_count / df.cell_type$cell_number.Freq,0)

In [12]:
write.csv(df.cell_type,
          "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/BMMC/1.prepare_data/3.merge_cell_type.240522/df.cell_type.csv")

In [14]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.9 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] ggplot2_3.5.1      magrittr_2.0.3     Signac_1.13.0      Seurat_5.0.3      
[5] SeuratObject_5.0.1 sp_2.1-4          

loaded via a namespace (and not atta